In [3]:
from langchain_community.chat_models import ChatLlamaCpp
%pip install flash_attn


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [20 lines of output]
  Traceback (most recent call last):
    File "C:\Users\Iheb\Desktop\projects\finetune_gemma\env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\Iheb\Desktop\projects\finetune_gemma\env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\Iheb\Desktop\projects\finetune_gemma\env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
      return hook(config_settings)
             ^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\Iheb\AppData\Local\Temp\pip-build-env-viflueas\overlay\Lib\site-packages\setuptools\build_meta.py", line 334


  Using cached flash_attn-2.7.0.post2.tar.gz (2.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


In [4]:
import torch 
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
# test.py
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

img_path=r"C:\Users\Iheb\Pictures\Screenshots\text before bin.png"
cache_dir = r"C:\Users\Iheb\.cache\huggingface\hub\models--openbmb--MiniCPM-V-2_6-int4\snapshots\6f1555d8e2359cb18595da9f1864cb41631e0617"


model = AutoModelForCausalLM.from_pretrained('openbmb/MiniCPM-V-2_6-int4',      
                                             local_files_only=True,
                                             trust_remote_code=True,
                                             attn_implementation="sdpa")
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6-int4',
                                          local_files_only=True,
                                          trust_remote_code=True)
model=model.eval()



Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model.config

MiniCPMVConfig {
  "_name_or_path": "openbmb/MiniCPM-V-2_6-int4",
  "architectures": [
    "MiniCPMV"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "openbmb/MiniCPM-V-2_6-int4--configuration_minicpm.MiniCPMVConfig",
    "AutoModel": "openbmb/MiniCPM-V-2_6-int4--modeling_minicpmv.MiniCPMV",
    "AutoModelForCausalLM": "openbmb/MiniCPM-V-2_6-int4--modeling_minicpmv.MiniCPMV"
  },
  "batch_vision_input": true,
  "bos_token_id": 151643,
  "drop_vision_last_layer": false,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "image_size": 448,
  "initializer_range": 0.02,
  "intermediate_size": 18944,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "minicpmv",
  "num_attention_heads": 28,
  "num_hidden_layers": 28,
  "num_key_value_heads": 4,
  "patch_size": 14,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint

In [5]:
img=Image.open(r"C:\Users\Iheb\Pictures\Screenshots\Screenshot 2024-11-26 140345.png").convert("RGB")
invoice_prompt = ("""
This image represent an invoice, your task is to retrieve information from this invoice in the format mentioned below /n

///FORMAT///
Account Number: {the account number}  
Receiver Name: {to WHOM THIS INVOICE IS SENT}  
Receiver Address: {the address of the customer}  
Amount to be Paid: {the amount paid}  
Period of the Invoice: {start period - end period}  

//
"""
)

job_requirements="""
Requirements: /n

Master’s degree or higher in Data Science, Computer Science, Computer Engineering, Electrical Engineering, Robotics, Physics, Mathematics, or related fields.
Deep understanding and experience with Image Theory, Deep Learning, Machine Learning, Python Programming, TensorFlow, or PyTorch.
Strong experience in designing and implementing large-scale data processing, analysis, and exploration solutions.
Proficiency in R, Python, advanced SQL skills, and a good understanding of data management principles.
Experience with cloud technologies (Azure or GCP preferred) and deploying CI/CD pipelines.
Knowledge of Git, PySpark, Data Bricks, and database management (SQL/PLSQL/Snowflake preferred).
Excellent communication skills and the ability to work independently and collaborate across teams.
2+ years of industry or academia experience in image processing, computer vision, or related fields."""

resume_prompt=(f"""
     "You are an AI recruiter tasked with selecting the best resume for a specific job opening. Carefully analyze the provided job description. Your goal is to evaluate the resume based on relevance, skills, experience, and alignment with the job.
     ///NOTE: YOU ARE ONLY DEMANDED TO RETURN THE OUTPUT FORMAT ///

///Job Description:///
{job_requirements}



///Evaluation Criteria:///

Skill Match: Rate how well the candidate’s skills align with the required skills (the section mentioned above)./n
Experience Relevance: Assess the relevance of the candidate's previous work experience to the job description./n
Education Fit: Evaluate the candidate's educational background in relation to the job requirements./n
Achievements: Identify and score notable achievements, certifications, or projects that demonstrate qualifications for this role./n
Overall Fit: Provide an overall score and explanation of why this candidate is the best fit.
Output Format:/n

///Output Format:////n
Generate the following output:

Candidate Name: [Name]
Skill Match Score: [0-10]
Experience Relevance Score: [0-10]
Education Fit Score: [0-10]
Notable Achievements: [List key achievements and minimize them ]
Overall Fit Score: [0-10]
Explanation: [Summarize why this candidate is or isn't the best fit for the role.]
""")


In [7]:
comments=("I hate this application, adds keep popping up and does not allow me to navigate the site","This application is slower than my grandma, you should fix this. I don't want to")

problem_template=("""
You are responsable for the customer satisfaction departement in the Entreprise and your job is to analyse the comments, highlight the problem and then suggest a solution for it./n/n

////Comments///
1. "I have been using this app for a while now and I must say that it has been nothing but frustrating. The interface is confusing, the features are limited, and the customer support is practically nonexistent."/n
2. "I had high hopes for this app, but it has completely failed to meet my expectations. It's slow, unreliable, and constantly crashes. I've tried reaching out to their support team multiple times, but they never seem to respond in a timely manner." /n
3. "The concept of this app seems interesting, but its execution leaves much to be desired. The design is outdated, the functionality is lacking, and the overall user experience is just terrible."/n
4. "I'm really disappointed with how this app has turned out. It promised so much, but delivered very little. Its lack of features and poor performance make it almost unusable."/n
5. "I regret ever downloading this app. It's filled with bugs, glitches, and errors that make it nearly impossible to use. The developers should seriously consider revamping it or giving up on it altogether."/n

///OUTPUT FORMAT:/n///
Return a json like format highlighting briefly the problem encountered and the solution for every comment and keep it short.
 
""")

In [8]:
comments_template="""Generate me 5 comments from dissatisfied customers using an application that you may specify"""

msgs = [{'role': 'user', 'content': [problem_template, None]}]


res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7,
    stream=True
)

generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')

C:\Users\Iheb\Desktop\projects\finetune_gemma\env\Lib\site-packages\transformers\models\auto\image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
C:\Users\Iheb\Desktop\projects\finetune_gemma\env\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:521: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{
  "comment1": {
    "problem": "Confusing interface, limited features, and poor customer support.",
    "solution": "Improve the app's user interface design, expand the feature set, and enhance customer support response times."
  },
  "comment2": {
    "problem": "Slowness, unreliability, crashes, and lack of timely support.",
    "solution": "Optimize app performance, fix bugs, and ensure a faster response from the support team."
  },
  "comment3": {
    "problem": "Outdated design, lacking functionality, and terrible user experience.",
    "solution": "Revamp the app's design, add new functionalities, and improve overall usability."
  },
  "comment4": {
    "problem": "Lack of features and poor performance making it almost unusable.",
    "solution": "Add more features to the app and optimize its performance for better user experience."
  },
  "comment5": {
    "problem": "Filled with bugs, glitches, errors, and nearly impossible to use.",
    "solution": "Fix existing bugs, improv

In [6]:
from transformers import 
lcd_img=Image.open(r"C:\Users\Iheb\Pictures\Screenshots\Screenshot 2024-11-27 215951.png")
hardware=("""you are a hardware expert with 30+ years of experience and you encountered a specific problem here is the statement of the problem. You are to provide a solution for this problem

//problem statement//:
dans mon système si j'ai éliminé la pompe et les 4 injecteurs le système fonctionne correctement et il donne l'affichage correcte et détaillé. si j'ai installé la pompe et les 4 injecteurs il fonctionne correctement et afficher comme il faut et j'ai appuyé sur le bouton marche il affiche le système en cours... mais si le cycle est arrêté l'afficheur LCD affiche des symbole et des zéros sur l'ecriture, donc le défaut ou dans la partie programmation ou la partie de câblage sachant que j'ai alimenté le systéme par une boitier d'alimentation de l'ordinateur qui fournis 12V 10A max( alimenter par les 4 injecteurs et la pompe), 5V 25A max(alimenter par la module de 5 relais et l'afficheur LCD 20x4) et 3.3V 14A max (alimenter par la carte ESP 32) et pour la boitier donne une masse commune qui relier tous les systèmes avec les boutons aussi. donc est ce que c'est possible tu me donner ou le defaut ?

"""
)

msgs = [{'role': 'user', 'content': [hardware, lcd_img]}]

res = model.chat(
    image=None,
    msgs=msgs,
    max_len=4096,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7,
    stream=True
)

generated_text = ""
for new_text in res:
    generated_text += new_text
    print(new_text, flush=True, end='')

Your problem appears to be related to the LCD display showing symbols and zeros instead of displaying data correctly when the system is stopped. Here are some possible solutions:

1. **Check the Power Supply:**
   Ensure that all power supplies (12V, 5V, 3.3V) are stable and within their respective maximum limits (10A, 25A, 14A). Voltage drops or instability can cause erratic behavior in the LCD display.

2. **Verify Connections:**
   Make sure all connections between the LCD display, ESP32 board, and other components are secure and properly made. Loose or incorrect connections can lead to display issues.

3. **Reset the System:**
   Sometimes a simple reset of the system can resolve display issues. Try resetting the ESP32 board by holding down the reset button for a few seconds while the system is powered on.

4. **Check the Firmware:**
   Ensure that the firmware running on the ESP32 board is up-to-date and compatible with your setup. You may need to update or replace the firmware if

In [13]:
model.__class__

transformers_modules.openbmb.MiniCPM-V-2_6-int4.6f1555d8e2359cb18595da9f1864cb41631e0617.modeling_minicpmv.MiniCPMV

In [ ]:
from langchain_huggingface import HuggingFacePipeline

hf_model = HuggingFacePipeline()
